In [7]:
import pandas as pd
import pickle
from utils import post_processing, video_to_tank, calculate_overlap
from scipy.spatial import distance

In [8]:
with open('data.pickle', 'rb') as f:
    data = pickle.load(f)

In [9]:
data_frame = pd.DataFrame({
    'init_place':[],
    'init_bottom_time':[],
    'total_dist':[],
    'mean_speed':[],
    'total_bottom_time':[],
    'freeze_count':[],
    'mean_freeze_time':[],
    'hyp_act_count':[],
    'hyp_act_total_time':[]}
)

In [11]:
for video in data:
    for i, v in enumerate(data[video]):
        dat, start = post_processing(v[1:])
        h, w = v[0]
        

        abs_dist = 0
        abs_speed = 0
        count = 0
        total_bottom_count = 0
        mod = 'up'
        down_count = 0
        freezes = []
        frez_count = 0

        hyp_act_ls = []
        hyp_act_count = 0
        for i in range(1, len(dat)):

            x, y = video_to_tank(dat[count], h, w)
            if y > 10:
                total_bottom_count += 1

            if count < (30 * 10) and mod == 'up':
                if y > 10:
                    mod = 'down'
                    down_count += 1
                if count == (30 * 10 - 1):
                    init = 0
                    mod = 'break'
            elif mod == 'down' and y > 10:
                init = 1
                down_count += 1
            else:
                mod = 'break'






            d = distance.euclidean(video_to_tank(dat[i-1], h, w), video_to_tank(dat[i], h, w))
            abs_dist += d
            abs_speed += d * 30
            count += 1


            if calculate_overlap(dat[i-1], dat[i]) > 0.95 and d <= 0.0165:
                frez_count += 1
            elif calculate_overlap(dat[i-1], dat[i]) > 0.95 and abs((video_to_tank(dat[i-1], h, w))[1] - (video_to_tank(dat[i], h, w))[1]) <= 0.0165:
                frez_count += 1
            else:
                freezes.append(frez_count)
                frez_count = 0


            if d * 30 > 10:
                hyp_act_count += 1
            else:
                hyp_act_ls.append(hyp_act_count)
                hyp_act_count = 0




        fris = len(freezes)
        fris_mean = sum(freezes) / len(freezes)
        init_bottom_time = down_count / 30
        total_bottom_time = total_bottom_count / 30
        mean_speed = abs_speed / count

        hyp_act_abs_time = sum(hyp_act_ls) / 30
        hyp_acts = len([i for i in hyp_act_ls if i != 0])



        data_dict = {
            'init_place':init,
            'init_bottom_time':init_bottom_time,
            'total_dist':abs_dist,
            'mean_speed':mean_speed,
            'total_bottom_time':total_bottom_time,
            'freeze_count':fris,
            'mean_freeze_time':fris_mean,
            'hyp_act_count': hyp_acts,
            'hyp_act_total_time': hyp_act_abs_time} 


        data_frame = pd.concat([data_frame, pd.DataFrame([data_dict])], ignore_index=True)




data_frame